In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from database import get_database , save_to_database
from extract_info import *
from config import *
from extract_detail import *


In [3]:
"""
Cấu hình kết nối MongoDB và các thông số liên quan đến việc thu thập dữ liệu phim từ CGV.
"""

MONGO_URL = "mongodb+srv://CgvHub:o@cluster0.ktuns.mongodb.net/"
DATABASE_NAME = "cgv_movies"

# Tên collection trong MongoDB
FILM_COLLECTION_NAME = "films"
NEW_OFFER_COLLECTION_NAME = "new_offers"
THEATERS_COLLECTION_NAME = "theaters"
# URL của các trang phim
NEW_OFFERS_URL = "https://www.cgv.vn/default/newsoffer/"
NOW_SHOWING_URL = "https://www.cgv.vn/default/movies/now-showing.html"
COMING_SOON_URL = "https://www.cgv.vn/default/movies/coming-soon-1.html"
THEATERS_URL = "https://www.cgv.vn/default/cinox/site/"
# Thời gian chờ (giây) giữa các lần thu thập dữ liệu
WAIT_TIME = 10

In [3]:
# Kết nối MongoDB
db = get_database()
if db is None:
    print("[❌] Kết nối cơ sở dữ liệu thất bại. Đang thoát...")

Kết nối thành công đến cơ sở dữ liệu: 'cgv_movies'


In [4]:
films_col = db[FILM_COLLECTION_NAME]
newoffers_col = db[NEW_OFFER_COLLECTION_NAME]
theaters_col = db[THEATERS_COLLECTION_NAME]

In [11]:
# Khởi tạo trình điều khiển Chrome
service = Service()
driver = webdriver.Chrome(service=service)

In [12]:
def extract_all_theaters_info(driver, url, theaters_col):
    try:
        driver.get(url)
        time.sleep(WAIT_TIME)  # Chờ trang tải xong, có thể thay bằng WebDriverWait nếu cần

        # Tìm tất cả các thành phố có thể chọn
        city_elements = driver.find_elements(By.CSS_SELECTOR, "span[id^='cgv_city_']")
        city_ids = [city.get_attribute("id") for city in city_elements]

        for city_id in city_ids:
            try:
                city_element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, city_id)))
                driver.execute_script("arguments[0].click();", city_element)
                print(f"🌆 Đang thu thập dữ liệu cho thành phố: {city_element.text.strip()}")
                time.sleep(WAIT_TIME)

                # Lấy danh sách rạp sau khi chọn thành phố
                theater_elements = driver.find_elements(By.CSS_SELECTOR, "span[id^='cgv_site_']")
                theater_ids = [theater.get_attribute("id") for theater in theater_elements if theater.is_displayed()]

                print("📌 Danh sách rạp có thể thu thập:", theater_ids)

                for theater_id in theater_ids:
                    try:
                        theater_element = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.ID, theater_id)))
                        driver.execute_script("arguments[0].click();", theater_element)
                        print(f"✅ Đang thu thập dữ liệu từ rạp: {theater_id}")
                        time.sleep(WAIT_TIME)

                        # Gọi hàm thu thập dữ liệu rạp
                        theater_info = extract_theater_info(driver)
                        if theater_info:
                            save_to_database(theaters_col, theater_info)
                            print(f"💾 Dữ liệu từ {theater_id} đã được lưu.")
                    except Exception as e:
                        print(f"⚠️ Lỗi khi thu thập dữ liệu từ rạp {theater_id}: {e}")

            except Exception as e:
                print(f"⚠️ Lỗi khi xử lý thành phố {city_id}: {e}")

    except Exception as e:
        print(f"🚨 Lỗi khi truy cập URL {url}: {e}")

In [13]:
extract_all_theaters_info(driver,THEATERS_URL,theaters_col)

🌆 Đang thu thập dữ liệu cho thành phố: Hồ Chí Minh
📌 Danh sách rạp có thể thu thập: ['cgv_site_004', 'cgv_site_008', 'cgv_site_021', 'cgv_site_023', 'cgv_site_024', 'cgv_site_030', 'cgv_site_022', 'cgv_site_035', 'cgv_site_006', 'cgv_site_010', 'cgv_site_012', 'cgv_site_031', 'cgv_site_033', 'cgv_site_034', 'cgv_site_038', 'cgv_site_057', 'cgv_site_071', 'cgv_site_073', 'cgv_site_079', 'cgv_site_088', 'cgv_site_101']
✅ Đang thu thập dữ liệu từ rạp: cgv_site_004
📅 Đang lấy dữ liệu cho ngày: 18-03-2025
📅 Đang lấy dữ liệu cho ngày: 19-03-2025
📅 Đang lấy dữ liệu cho ngày: 20-03-2025
⚠️ Ngày 20-03-2025 không có dữ liệu. Số ngày trống liên tiếp: 1
📅 Đang lấy dữ liệu cho ngày: 21-03-2025
📅 Đang lấy dữ liệu cho ngày: 22-03-2025
📅 Đang lấy dữ liệu cho ngày: 23-03-2025
📅 Đang lấy dữ liệu cho ngày: 24-03-2025
📅 Đang lấy dữ liệu cho ngày: 25-03-2025
📅 Đang lấy dữ liệu cho ngày: 26-03-2025
📅 Đang lấy dữ liệu cho ngày: 27-03-2025
📅 Đang lấy dữ liệu cho ngày: 28-03-2025
⚠️ Ngày 28-03-2025 không có dữ

In [9]:
extract_film_info(driver, NOW_SHOWING_URL,films_col)


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF7995DFE45+26629]
	(No symbol) [0x00007FF799546010]
	(No symbol) [0x00007FF7993D914C]
	(No symbol) [0x00007FF79941F75F]
	(No symbol) [0x00007FF799457972]
	(No symbol) [0x00007FF7994522CE]
	(No symbol) [0x00007FF799451379]
	(No symbol) [0x00007FF7993A55E5]
	GetHandleVerifier [0x00007FF7999372BD+3529853]
	GetHandleVerifier [0x00007FF79994DA22+3621858]
	GetHandleVerifier [0x00007FF7999424D3+3575443]
	GetHandleVerifier [0x00007FF7996AB77A+860474]
	(No symbol) [0x00007FF79955088F]
	(No symbol) [0x00007FF7993A41FF]
	GetHandleVerifier [0x00007FF7999BECD8+4085400]
	BaseThreadInitThunk [0x00007FF80BA6259D+29]
	RtlUserThreadStart [0x00007FF80D00AF38+40]


In [ ]:
extract_film_info(driver, COMING_SOON_URL,films_col)

In [8]:
extract_newoffer_info(driver,NEW_OFFERS_URL,newoffers_col)

Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi trích xuất phim: 'title'
Lỗi khi tr

KeyboardInterrupt: 

In [ ]:
driver.quit()
print("[🛑] Trình duyệt đã được đóng.")